In [ ]:
import os
import numpy as np
import pandas as pd

def get_weights(path, circle, type, holder):
    data = pd.read_csv(f'{path}\circle{circle}.txt', header=None)

    type_col = data[3]
    weight_col = 4
    holder_col = data[5].astype(int)
    
    filtered_data = data[(type_col == type) & (holder_col == holder)]

    return np.array(filtered_data[weight_col])*2.0

def get_weights_filtered(path, circle, type, holder, window_size=2):
    data = pd.read_csv(f'{path}\circle{circle}.txt', header=None)    

    type_col = data[3]
    weight_col = 4
    holder_col = data[5].astype(int)
    
    filtered_data = data[(type_col == type) & (holder_col == holder)]
    
    weights = np.array(filtered_data[weight_col])
    
    # Фильтрация шума с использованием скользящего среднего
    if len(weights) >= window_size:
        filtered_weights = np.convolve(weights, np.ones(window_size), 'valid') / window_size
    else:
        filtered_weights = weights
    
    return filtered_weights*2.0

def calculate_weights(path, type):
    dfs = []

    for circle in range(2, 32):
        data = []
        
        for holder in range(368):
            mean_weight = get_weights_filtered(path, circle=circle, type=type, holder=holder).mean()
            
            data.append({"Holder": holder, f"circle{circle}": mean_weight})

        dfs.append(pd.DataFrame(data))

    df = pd.concat(dfs, axis=1)

    # Удаляем дубликаты столбцов "Holder"
    df = df.loc[:,~df.columns.duplicated()]

    # Добавляем новые индексы в DataFrame
    for index in ["среднее", "σ"] + [f"Δ{i}" for i in range(2, 32)] + [">2σ", ">3σ"]:
        df[index] = ""

    file_path = os.path.join(path, f'weights_{type}.xlsx')

    df.to_excel(file_path, index=False)

In [ ]:
%%time
calculate_weights("без 0.1 смещений\offset=0.1_discard=1", "B")